# 데이터 가져오기

In [1]:
import pandas as pd

data = {
    '키': [150, 160, 170, 175, 165, 155, 172, 168, 174, 158,
          162, 173, 156, 159, 167, 163, 171, 169, 176, 161],
    '몸무게': [42, 50, 70, 64, 56, 48, 68, 60, 65, 52,
            54, 67, 49, 51, 58, 55, 69, 61, 66, 53]
}
df = pd.DataFrame(data)
df

,키,몸무게
0,150,42
1,160,50
2,170,70
3,175,64
4,165,56
5,155,48
6,172,68
7,168,60
8,174,65
9,158,52


In [2]:
# OLS 확인
# y값이 연속적으로 나오는 상황에서 추천
# Ordinary Least Squares (최소제곱법)
# 데이터의 실제값과 예측값의 차이(잔차)의 제곱합이 최소가 되도록 회귀선(선형식)을 찾는 통계적 방법

In [3]:
from statsmodels.formula.api import ols
model = ols('키 ~ 몸무게', data=df).fit() # '키'를 '몸무게'로부터 예측하겠다
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      키   R-squared:                       0.892
Model:                            OLS   Adj. R-squared:                  0.886
Method:                 Least Squares   F-statistic:                     148.0
Date:                Fri, 10 Oct 2025   Prob (F-statistic):           4.04e-10
Time:                        01:24:08   Log-Likelihood:                -45.761
No. Observations:                  20   AIC:                             95.52
Df Residuals:                      18   BIC:                             97.51
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    115.0676      4.158     27.671      0.0

In [4]:
# 결정계수값 확인
# : 모델이 데이터를 얼마나 잘 설명하는지를 나타내는 점수
model.rsquared

0.8915914350087264

In [6]:
# 기울기 /절편 확인
print("기울기 : ", model.params['몸무게']) # 베타1 # 몸무게(x)가 1kg 증가 -> 키 0.86cm 증가
print("절편 : ", model.params['Intercept']) # 베타0 # 몸무게가 0kg -> 키 115cm

기울기 :  0.8658438852380201
절편 :  115.06763904471865


In [7]:
# p_value 확인 : 관계여부에 대한 확률
# 관계정도 확률 / 0.05 미만 - 유의함(관계있음) / 0.05 이상 - 유의 x(관계가 없음) )
print("pvalue:", model.pvalues['몸무게'])
print("pvalue:", "{:.10f}".format(model.pvalues['몸무게']))

pvalue: 4.0379325599303615e-10
pvalue: 0.0000000004


In [ ]:
# pvalue: 4.0379325599303615e-10 # 0.00000000004 - 0.05보다 작다 -> 관계 있을 확률 높음 #몸무게(x) - 키(y) 영향준다
# pvalue: 0.0000000004

In [9]:
# 잔차확인
# 잔차 : 결과의 오차
# : 표본(sample)으로 추정한 회귀식과 실제 관측값의 차이
df['잔차'] = df['키'] - model.predict(df)
df

,키,몸무게,잔차
0,150,42,-1.433082
1,160,50,1.640167
2,170,70,-5.676711
3,175,64,4.518352
4,165,56,1.445103
5,155,48,-1.628146
6,172,68,-1.945023
7,168,60,0.981728
8,174,65,2.652508
9,158,52,-2.091521


In [11]:
# 몸무게(x)가 67kg일 때 예측 키(y)

new_data = pd.DataFrame({'몸무게':[67]})
new_data
result = model.predict(new_data)
result

0    173.079179
dtype: float64

In [15]:
# 신뢰구간 확인
print("신뢰구간:\n", model.conf_int(alpha=0.05).loc['몸무게'])
# alpha : 95% 신뢰구간에 한정해서 몸무게 구간의 사이값(기울기)확인

신뢰구간:
 0    0.716337
1    1.015351
Name: 몸무게, dtype: float64


In [ ]:
# 기울기값 0.71 ~ 1.0 사이(95%정도의 신뢰)
0.716337
~
1.015351

In [17]:
new_data = pd.DataFrame({"몸무게": [50]})
pred = model.get_prediction(new_data)
result = pred.summary_frame(alpha=0.05) # 95%의 신뢰도 지님
result
# print("예측값의 신뢰구간과 예측구간:\n", result)

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,158.359833,0.794986,156.68963,160.030037,152.820798,163.898869


In [12]:
# 잔차 제곱합(오차 전체 크기)
SSR=sum(df['잔차']**2) 
SSR

113.74226638884431

In [13]:
# MSE(잔차제곱의 평균)
MSE = (df['잔차']**2).mean()
MSE

5.687113319442214

In [14]:
from sklearn.metrics import mean_squared_error
pred = model.predict(df['몸무게'])
mse = mean_squared_error(df['키'], pred)
print("MSE :", mse)

MSE : 5.687113319442214


In [ ]:
# <<결과값>>

# 설명해주는 메타 데이터 값
#                             OLS Regression Results                            
# ==============================================================================
# Dep. Variable(종속변수 y값):        키   R-squared(정확도):               0.892(모델의 설명정도 / 89% 설명력이 매우 높음)
# Model(설명지표모델):               OLS   Adj. R-squared:                  0.886(모델의 설명정도 / x값이 많을 때도 적용되는 설명정도)
# Method:                 Least Squares   F-statistic:                     148.0(F검정:모델 전체 유의성 검정, 값이 클수록 모델이 적합하다)
# Date:                Fri, 10 Oct 2025   Prob (F-statistic):           4.04e-10(F검정의 p값 : 값이 작을수록 유의함(0.05이하)
# Time:                        01:24:08   Log-Likelihood:                -45.761(로그우도값 : 모델의 적합도 지표/ 절대값 작을수록 좋음)
# No. Observations(데이터개수):       20   AIC:                             95.52(모델 품질 지표/ 낮을수록 좋음)
# Df Residuals(잔차자유도/데이터수(20)-회귀계수(2): 18   BIC:                97.51(베이지안 정보 기준 / 낮을수록 좋음)
# Df Model:                           1(독립변수, x값, 현재는 몸무게)                                         
# Covariance Type:            nonrobust(공분산 계산 방식 : 기본값(등분산 가정) / 공분산 : 변수들이 같이 변하는 정도 / 등분산: 븐신정도가 같다                                         
# ==============================================================================

# ★★★중요 포인트 ★★★
# 결과값(하단부)
#                  coef    std err          t      P>|t|      [0.025      0.975] - 신뢰도 : 0.975 - 0.025 = 95%
# ------------------------------------------------------------------------------
# Intercept    115.0676      4.158     27.671      0.000     106.331     123.804 = (범위 : 106.331 ~ 123.804)
# 몸무게            0.8658      0.071     12.167      0.000       0.716       1.015
# ==============================================================================
# Omnibus:                        0.985   Durbin-Watson:                   2.609
# Prob(Omnibus):                  0.611   Jarque-Bera (JB):                0.336
# Skew:                          -0.315   Prob(JB):                        0.845
# Kurtosis:                       3.082   Cond. No.                         432.
# ==============================================================================

In [ ]:
# ======================= 회귀분석 주요 지표 설명 =======================

# ① coef (회귀계수) - β₁(기울기)
# → 독립변수가 1단위 증가할 때 종속변수가 얼마나 변하는지를 의미함.
# 예: coef = 0.8658 → 몸무게가 1 증가할 때 키가 평균 0.8658 증가한다는 뜻.

# ② std err (표준오차, Standard Error)
# → 추정된 회귀계수(coef)가 얼마나 불확실한지를 보여주는 값.
# → 값이 작을수록 추정이 안정적이고, 계수가 신뢰할 만함을 의미함.

# ③ t (t-통계량)
# → 회귀계수가 ‘0이 아니다’라는 가설을 검증하는 통계량.
# → t값 = coef / std err 로 계산됨.
# → 절댓값이 클수록 독립변수가 종속변수에 유의미한 영향을 미친다는 뜻.

# ④ P>|t| (p값, 유의확률)
# → t검정의 p값. "이 변수가 실제로 영향이 없을 확률"을 의미함.
# → p < 0.05 → 통계적으로 유의함 (즉, 실제로 영향이 있다).
# → p ≥ 0.05 → 통계적으로 유의하지 않음 (영향이 없을 가능성 큼).

# ⑤ [0.025  0.975] (95% 신뢰구간)
# → 회귀계수(coef)가 95% 확률로 포함될 범위.
# → 예: [0.716, 1.015] → 계수는 0.716~1.015 사이일 가능성이 95%.
# → 이 구간에 0이 포함되지 않으면, 그 변수는 유의하다고 볼 수 있음.

# ======================= 한 줄 요약 =======================
# coef : 영향의 크기(기울기)
# std err : 계수의 불확실성
# t : 영향의 통계적 강도
# P>|t| : 영향의 유의성(p<0.05면 유의)
# [0.025 0.975] : 계수가 존재할 95% 신뢰구간

# 1. OLS란?
OLS는 최소제곱법(Ordinary Least Squares) 으로,
데이터의 실제값과 예측값의 차이(잔차) 의 제곱합이 최소가 되도록
회귀선(선형식) 을 찾는 통계적 방법이다.

[ Y = β₀ + β₁X + ε ]

Y: 종속변수 (예: 키)
X: 독립변수 (예: 몸무게)
β₀: 절편(intercept)
β₁: 기울기(slope)
ε: 오차항(residual) - 예측하지 못한 오류
OLS는 다음 식을 최소화하도록 β₀, β₁을 찾는다.

# 회귀분석 기본 개념 요약표

| 구분 | 용어 | 뜻 (쉽게 설명) | 예시 |
|------|------|----------------|------|
| 1 | 회귀분석 (Regression) | 한 변수(Y)가 다른 변수(X)에 따라 어떻게 변하는지 관계를 찾는 방법 | 몸무게가 늘면 키도 커질까? |
| 2 | 절편 (Intercept, β₀) | X=0일 때 Y의 예측값. 그래프가 y축과 만나는 점 | 몸무게 0kg일 때 키 약 115cm |
| 3 | 회귀계수 (Coefficient, β₁) | X가 1 증가할 때 Y가 얼마나 변하는지. 그래프의 기울기 | 몸무게 1kg 늘면 키 0.87cm 증가 |
| 4 | 잔차 (Residual) | 실제값 - 예측값 (오차) | 실제 키 170, 예측 168 → 잔차 = 2 |
| 5 | 결정계수 (R²) | 모델이 데이터를 얼마나 잘 설명하는가 (0~1) | R²=0.89 → 89% 설명 가능 |
| 6 | p-value (유의확률) | 변수가 Y에 진짜 영향을 주는지 판단 | p<0.05 → 진짜 영향 있음 |
| 7 | F통계량 (F-statistic) | 전체 모델이 유의한지 검사 | F값 큼 + p<0.05 → 모델 유의함 |
| 8 | Durbin-Watson | 오차 간의 상관 확인 (2에 가까울수록 좋음) | DW=2.0 → 잔차가 독립적 |
| 9 | 정규성 검정 (Omnibus, JB) | 오차들이 무작위로 분포하는지 확인 | p>0.05 → 정규성 만족 |
| 10 | 예측식 예시 | 회귀식 형태로 표현 | 키 = 115.07 + 0.87 × 몸무게 |